<a href="https://colab.research.google.com/github/Justin-Hwang/EEG-AD-FTD-Detection/blob/main/FTD_CN_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 그중에 2025 Lab Research 폴더 안을 확인
!ls "/content/drive/MyDrive/2025_Lab_Research"

import sys
sys.path.append('/content/drive/MyDrive/2025_Lab_Research')

import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Running on", DEVICE)  # → “cuda” 가 뜨면 GPU 정상

'AD vs FTD vs CN Binary Classification'   eeg_holdout-5.db
'Binary Classification LOSO Test.ipynb'   eeg_holdout-6.db
'Colab Files'				  eeg_holdout-7.db
'Data Preparation.gdoc'			  eeg_holdout-8.db
 eeg_dataset.py				  eeg_holdout-9.db
 EEGformer_model_training.ipynb		  eeg_holdout.db
 eegformer_optuna_cv_3.db		  eeg_holdout_fixed_1.db
 eegformer_optuna_cv_4.db		  eeg_optuna_trial_1.db
 eegformer_optuna_cv_5.db		  eeg_optuna_trial_2.db
 eeg_grid_search-10.db			  eeg_optuna_trial_3.db
 eeg_grid_search-11.db			 'EEG Transformer Architecture.gdoc'
 eeg_grid_search-12.db			 'Lab Info'
 eeg_grid_search-13.db			 'Lab Research Final Report'
 eeg_grid_search-14.db			 'Lab Research Paper Review'
 eeg_grid_search-15.db			 'Meeting Note.gdoc'
 eeg_grid_search-16.db			  model-data
 eeg_grid_search-17.db			  model-data.zip
 eeg_grid_search-18.db			  model_optimized_2.py
 eeg_grid_search-2.db			  model_optimized_3.py
 eeg_grid_search-3.db			  model_optimized_4.py
 eeg_grid_search-4.db			  model_opt

In [ ]:
import wandb
wandb.login()  # 첫 실행 시 API 키 입력

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jh8032 (jh8032-new-york-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
!pip install optuna
!pip install wandb
!pip install mne

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 65.7 MB/s eta 0:00:00


#### FTD vs CN (Search the Best Hyperparameter)

#### Baseline Results (Machine Learning) FTD vs CN
- LightGBM
- SVM
- KNN
- MLP
- Random Forests

In [ ]:
!pip install lightgbm scikit-learn

In [ ]:
# baseline_models.py  – Classical baselines for FTD vs CN
# -------------------------------------------------------
import os, json, random, time, copy
from pathlib import Path

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from sklearn.pipeline       import Pipeline
from sklearn.preprocessing  import StandardScaler
from sklearn.metrics        import (
    recall_score, f1_score, accuracy_score, confusion_matrix
)

from sklearn.svm            import SVC
from sklearn.neighbors      import KNeighborsClassifier
from sklearn.ensemble       import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from lightgbm               import LGBMClassifier   # pip install lightgbm

from eeg_dataset import EEGDataset                 # ← your existing loader

# ─── Reproducibility ────────────────────────────────────────────
SEED = 0
random.seed(SEED)
np.random.seed(SEED)

# ─── Paths & constants (adjust as needed) ───────────────────────
DATA_DIR   = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE = 'labels.json'
NUM_WORKERS = 4               # DataLoader workers

# ─── Dataset: returns flat NumPy feature vector + label ─────────
class FlatEEGDataset(Dataset):
    """
    Wraps EEGDataset but outputs (1‑D float32 NumPy array, int label).
    Works whether EEGDataset returns a tensor or an ndarray.
    """
    def __init__(self, raw_ds, metas, label_map):
        self.raw_ds = raw_ds
        self.labels = [label_map[d['label']] if isinstance(d['label'], str)
                       else d['label'] for d in metas]

    def __len__(self): return len(self.raw_ds)

    def __getitem__(self, idx):
        x, _ = self.raw_ds[idx]                       # tensor OR ndarray
        if isinstance(x, torch.Tensor):
            x = x.flatten().cpu().numpy()
        else:  # already ndarray
            x = x.flatten()
        return x.astype(np.float32), self.labels[idx]

# ─── Utility functions ──────────────────────────────────────────
def count_labels(meta, cls0='F', cls1='C'):
    n0 = sum(1 for d in meta if d['label'] in (cls0, 0))
    n1 = len(meta) - n0
    return n0, n1

def metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    spec = tn / (tn + fp) if (tn + fp) else 0.0
    return {
        'acc':  accuracy_score(y_true, y_pred),
        'rec':  recall_score(y_true, y_pred, zero_division=0),  # sensitivity
        'spec': spec,
        'f1':   f1_score(y_true, y_pred, zero_division=0)
    }

def report(name, res):
    print(f"{name:>11s}: "
          f"Accuracy={res['acc']:.4f} "
          f"Sensitivity={res['rec']:.4f} "
          f"Specificity={res['spec']:.4f} "
          f"F1={res['f1']:.4f}")

# ─── Load label metadata (same JSON your EEGformer uses) ────────
with open(Path(DATA_DIR) / LABEL_FILE, 'r') as f:
    all_meta = json.load(f)

train_meta       = [d for d in all_meta if d['type'] == 'train']
test_within_meta = [d for d in all_meta if d['type'] == 'test_within']
test_cross_meta  = [d for d in all_meta if d['type'] == 'test_cross']

class0, class1   = 'F', 'C'            # FTD vs CN
label_map        = {class0: 0, class1: 1}

# Keep only F & C in evaluation splits
test_within_meta = [d for d in test_within_meta if d['label'] in (class0, class1)]
test_cross_meta  = [d for d in test_cross_meta  if d['label'] in (class0, class1)]

# ─── Balance training split (down‑sample larger class) ──────────
data0 = [d for d in train_meta if d['label'] == class0]
data1 = [d for d in train_meta if d['label'] == class1]
k     = min(len(data0), len(data1))
balanced_meta = random.sample(data0, k) + random.sample(data1, k)
random.shuffle(balanced_meta)

for d in balanced_meta:
    d['label'] = label_map[d['label']]     # convert to int

print(f"\n[Balanced] TRAIN  total={len(balanced_meta)} (FTD={k}, CN={k})")

# ─── Convert each split to NumPy (X, y) -------------------------
def make_numpy(metas):
    raw = EEGDataset(DATA_DIR, metas)
    ds  = FlatEEGDataset(raw, metas, label_map)
    loader = DataLoader(ds, batch_size=128, shuffle=False,
                        num_workers=NUM_WORKERS)
    X, y = [], []
    for xb, yb in loader:          # xb is *tensor* batch
        X.append(xb)               # keep on CPU
        y.append(yb)
    return torch.cat(X).numpy(), torch.cat(y).numpy()

X_train, y_train = make_numpy(balanced_meta)
X_tw,    y_tw    = make_numpy(test_within_meta)
X_tc,    y_tc    = make_numpy(test_cross_meta)

print(f"Input feature vector length per sample = {X_train.shape[1]}")

# ─── Define classical models ------------------------------------
models = {
    "LightGBM" : LGBMClassifier(
        n_estimators=100, learning_rate=0.001,
        random_state=SEED, class_weight='balanced'
    ),
    "SVM"      : SVC(
        kernel='rbf', probability=False,
        class_weight='balanced', random_state=SEED, gamma='scale'
    ),
    "KNN"      : KNeighborsClassifier(
        n_neighbors=7, weights='distance'
    ),
    "MLP"      : MLPClassifier(
        hidden_layer_sizes=(256, 128), max_iter=200,
        activation='relu', solver='adam', random_state=SEED
    ),
    "RandomForest": RandomForestClassifier(
        n_estimators=100, max_depth=None,
        random_state=SEED, class_weight='balanced'
    )
}

# ─── Train & evaluate -------------------------------------------
print("\n=== Final Evaluation on Test Sets (Classical Baselines) ===")
for name, clf in models.items():
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('clf',    clf)
    ])

    t0 = time.time()
    pipe.fit(X_train, y_train)
    train_time = time.time() - t0

    y_pred_tw = pipe.predict(X_tw)
    y_pred_tc = pipe.predict(X_tc)

    res_tw = metrics(y_tw, y_pred_tw)
    res_tc = metrics(y_tc, y_pred_tc)

    print(f"\n-- {name} (train time {train_time:.1f}s) --")
    print(f"   test_within  total={len(y_tw)}")
    report("test_within", res_tw)
    print(f"   test_cross   total={len(y_tc)}")
    report("test_cross",  res_tc)


[Balanced] TRAIN  total=1458 (FTD=729, CN=729)
Input feature vector length per sample = 27075

=== Final Evaluation on Test Sets (Classical Baselines) ===


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 729, number of negative: 729
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.948458 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6904125
[LightGBM] [Info] Number of data points in the train set: 1458, number of used features: 27075
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



-- LightGBM (train time 70.6s) --
   test_within  total=198
test_within: Accuracy=0.4899 Sensitivity=0.5159 Specificity=0.4444 F1=0.5628
   test_cross   total=554
 test_cross: Accuracy=0.4892 Sensitivity=0.4756 Specificity=0.5061 F1=0.5078

-- SVM (train time 17.2s) --
   test_within  total=198
test_within: Accuracy=0.5202 Sensitivity=0.4841 Specificity=0.5833 F1=0.5622
   test_cross   total=554
 test_cross: Accuracy=0.4964 Sensitivity=0.4723 Specificity=0.5263 F1=0.5097

-- KNN (train time 0.5s) --
   test_within  total=198
test_within: Accuracy=0.5455 Sensitivity=0.5159 Specificity=0.5972 F1=0.5909
   test_cross   total=554
 test_cross: Accuracy=0.5235 Sensitivity=0.5147 Specificity=0.5344 F1=0.5448

-- MLP (train time 27.3s) --
   test_within  total=198
test_within: Accuracy=0.4545 Sensitivity=0.4127 Specificity=0.5278 F1=0.4906
   test_cross   total=554
 test_cross: Accuracy=0.5090 Sensitivity=0.5212 Specificity=0.4939 F1=0.5405

-- RandomForest (train time 25.2s) --
   test_withi

In [ ]:
# baseline_multiclass.py – Classical baselines for AD vs FTD vs CN
# -------------------------------------------------------
import os, json, random, time
from pathlib import Path

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from sklearn.pipeline       import Pipeline
from sklearn.preprocessing  import StandardScaler
from sklearn.metrics        import (
    recall_score, f1_score, accuracy_score, confusion_matrix
)

from sklearn.svm            import SVC
from sklearn.neighbors      import KNeighborsClassifier
from sklearn.ensemble       import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from lightgbm               import LGBMClassifier

from eeg_dataset import EEGDataset

# ─── Reproducibility ────────────────────────────────────────────
SEED = 0
random.seed(SEED)
np.random.seed(SEED)

# ─── Paths & constants ──────────────────────────────────────────
DATA_DIR   = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE = 'labels.json'
NUM_WORKERS = 4

# ─── Dataset wrapper ────────────────────────────────────────────
class FlatEEGDataset(Dataset):
    def __init__(self, raw_ds, metas, label_map):
        self.raw_ds = raw_ds
        self.labels = [label_map[d['label']] if isinstance(d['label'], str)
                       else d['label'] for d in metas]

    def __len__(self): return len(self.raw_ds)

    def __getitem__(self, idx):
        x, _ = self.raw_ds[idx]
        x = x.flatten().cpu().numpy() if isinstance(x, torch.Tensor) else x.flatten()
        return x.astype(np.float32), self.labels[idx]

# ─── Metrics function for multiclass ────────────────────────────
def metrics(y_true, y_pred, labels):
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    specificity_list = []

    for i in range(len(labels)):
        tp = cm[i, i]
        fn = cm[i, :].sum() - tp
        fp = cm[:, i].sum() - tp
        tn = cm.sum() - (tp + fp + fn)
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0
        specificity_list.append(specificity)

    return {
        'acc':  accuracy_score(y_true, y_pred),
        'rec':  recall_score(y_true, y_pred, average='macro', zero_division=0),
        'spec': np.mean(specificity_list),
        'f1':   f1_score(y_true, y_pred, average='macro', zero_division=0)
    }

def report(name, res):
    print(f"{name:>11s}: Accuracy={res['acc']:.4f} "
          f"Sensitivity={res['rec']:.4f} "
          f"Specificity={res['spec']:.4f} "
          f"F1={res['f1']:.4f}")

# ─── Load and label metadata ────────────────────────────────────
with open(Path(DATA_DIR) / LABEL_FILE, 'r') as f:
    all_meta = json.load(f)

label_map = {'A': 0, 'C': 1, 'F': 2}
train_meta       = [d for d in all_meta if d['type'] == 'train']
test_within_meta = [d for d in all_meta if d['type'] == 'test_within']
test_cross_meta  = [d for d in all_meta if d['type'] == 'test_cross']

for d in train_meta + test_within_meta + test_cross_meta:
    if isinstance(d['label'], str):
        d['label'] = label_map[d['label']]

print(f"[Multiclass] TRAIN total={len(train_meta)} | "
      f"AD={sum(d['label']==0 for d in train_meta)}, "
      f"CN={sum(d['label']==1 for d in train_meta)}, "
      f"FTD={sum(d['label']==2 for d in train_meta)}")

# ─── Convert to NumPy (X, y) ─────────────────────────────────────
def make_numpy(metas):
    raw = EEGDataset(DATA_DIR, metas)
    ds = FlatEEGDataset(raw, metas, label_map)
    loader = DataLoader(ds, batch_size=128, shuffle=False, num_workers=NUM_WORKERS)
    X, y = [], []
    for xb, yb in loader:
        X.append(xb)
        y.append(yb)
    return torch.cat(X).numpy(), torch.cat(y).numpy()

X_train, y_train = make_numpy(train_meta)
X_tw,    y_tw    = make_numpy(test_within_meta)
X_tc,    y_tc    = make_numpy(test_cross_meta)

print(f"Input feature vector length per sample = {X_train.shape[1]}")

# ─── Define baseline models ─────────────────────────────────────
models = {
    "LightGBM" : LGBMClassifier(
        n_estimators=100, learning_rate=0.001,
        random_state=SEED, class_weight='balanced'
    ),
    "SVM"      : SVC(
        kernel='rbf', probability=False,
        class_weight='balanced', random_state=SEED, gamma='scale'
    ),
    "KNN"      : KNeighborsClassifier(
        n_neighbors=7, weights='distance'
    ),
    "MLP"      : MLPClassifier(
        hidden_layer_sizes=(256, 128), max_iter=200,
        activation='relu', solver='adam', random_state=SEED
    ),
    "RandomForest": RandomForestClassifier(
        n_estimators=100, max_depth=None,
        random_state=SEED, class_weight='balanced'
    )
}

# ─── Train and evaluate ─────────────────────────────────────────
print("\n=== Final Evaluation on Test Sets (Classical Multiclass Baselines) ===")
for name, clf in models.items():
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('clf',    clf)
    ])

    t0 = time.time()
    pipe.fit(X_train, y_train)
    train_time = time.time() - t0

    y_pred_tw = pipe.predict(X_tw)
    y_pred_tc = pipe.predict(X_tc)

    res_tw = metrics(y_tw, y_pred_tw, labels=[0, 1, 2])
    res_tc = metrics(y_tc, y_pred_tc, labels=[0, 1, 2])

    print(f"\n-- {name} (train time {train_time:.1f}s) --")
    print(f"   test_within  total={len(y_tw)}")
    report("test_within", res_tw)
    print(f"   test_cross   total={len(y_tc)}")
    report("test_cross",  res_tc)


[Multiclass] TRAIN total=3219 | AD=1388, CN=1102, FTD=729
Input feature vector length per sample = 27075

=== Final Evaluation on Test Sets (Classical Multiclass Baselines) ===


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.120448 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6904125
[LightGBM] [Info] Number of data points in the train set: 3219, number of used features: 27075
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



-- LightGBM (train time 228.9s) --
   test_within  total=344
test_within: Accuracy=0.3547 Sensitivity=0.3402 Specificity=0.6715 F1=0.3390
   test_cross   total=873
 test_cross: Accuracy=0.3276 Sensitivity=0.3214 Specificity=0.6610 F1=0.3207

-- SVM (train time 104.6s) --
   test_within  total=344
test_within: Accuracy=0.3517 Sensitivity=0.3422 Specificity=0.6673 F1=0.3397
   test_cross   total=873
 test_cross: Accuracy=0.3219 Sensitivity=0.3182 Specificity=0.6572 F1=0.3176

-- KNN (train time 1.0s) --
   test_within  total=344
test_within: Accuracy=0.4215 Sensitivity=0.3737 Specificity=0.6878 F1=0.3664
   test_cross   total=873
 test_cross: Accuracy=0.3528 Sensitivity=0.3379 Specificity=0.6678 F1=0.3272

-- MLP (train time 63.3s) --
   test_within  total=344
test_within: Accuracy=0.4041 Sensitivity=0.3548 Specificity=0.6846 F1=0.3420
   test_cross   total=873
 test_cross: Accuracy=0.3562 Sensitivity=0.3410 Specificity=0.6722 F1=0.3269

-- RandomForest (train time 71.2s) --
   test_wit

In [ ]:
import os
import json
import random
import time
import copy

import numpy as np
import torch
import torch.nn as nn
import wandb
import optuna

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from torch.utils.data import DataLoader, Subset, Dataset

from eeg_dataset import EEGDataset
from model_optimized_6 import EEGformer

# ─── Reproducibility ────────────────────────────────────────────
SEED = 0
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark     = False

# ─── Configuration ───────────────────────────────────────────────
DATA_DIR    = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE  = 'labels.json'
DEVICE      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

MAX_EPOCHS  = 150
BATCH_SIZE  = 32
NUM_WORKERS = 4
ES_PATIENCE = 15

# ─── Fixed Hyper-parameters ───────────────────────────────────────
PCT_START   = 0.2  # fixed

# ─── Dataset wrapper ─────────────────────────────────────────────
class BinaryEEGDataset(Dataset):
    def __init__(self, raw_ds, metas):
        self.raw_ds = raw_ds
        self.metas  = metas
    def __len__(self):
        return len(self.raw_ds)
    def __getitem__(self, idx):
        x, _ = self.raw_ds[idx]
        y     = self.metas[idx]['label']
        return x, y

# ─── Load & count splits ──────────────────────────────────────────
with open(os.path.join(DATA_DIR, LABEL_FILE), 'r') as f:
    all_meta = json.load(f)

train_meta       = [d for d in all_meta if d['type']=='train']
test_within_meta = [d for d in all_meta if d['type']=='test_within']
test_cross_meta  = [d for d in all_meta if d['type']=='test_cross']

class0, class1 = 'F','C'
label_map = {class0:0, class1:1}

# F vs C Filtering
train_meta       = [d for d in train_meta if d['label'] in (class0, class1)]
test_within_meta = [d for d in test_within_meta if d['label'] in (class0, class1)]
test_cross_meta  = [d for d in test_cross_meta  if d['label'] in (class0, class1)]

def count_labels(meta_list):
    cnt0 = cnt1 = 0
    for d in meta_list:
        lbl = d['label']
        if isinstance(lbl, str):
            if lbl == class0: cnt0 += 1
            elif lbl == class1: cnt1 += 1
        else:
            if lbl == 0: cnt0 += 1
            elif lbl == 1: cnt1 += 1
    return cnt0, cnt1

n_tr0, n_tr1 = count_labels(train_meta)
n_tw0, n_tw1 = count_labels(test_within_meta)
n_tc0, n_tc1 = count_labels(test_cross_meta)

print(f"--> Data counts before balancing:")
print(f"    TRAIN         total={len(train_meta)}  F(TD)={n_tr0}, C(N)={n_tr1}")
print(f"    TEST_WITHIN   total={len(test_within_meta)}  F(TD)={n_tw0}, C(N)={n_tw1}")
print(f"    TEST_CROSS    total={len(test_cross_meta)}  F(TD)={n_tc0}, C(N)={n_tc1}\n")

# ─── Balance train set ───────────────────────────────────────────
data0 = [d for d in train_meta if d['label']==class0]
data1 = [d for d in train_meta if d['label']==class1]
min_count = min(len(data0), len(data1))

balanced_meta = random.sample(data0, min_count) + random.sample(data1, min_count)
balanced_meta = [copy.deepcopy(d) for d in balanced_meta]
random.shuffle(balanced_meta)

# ─── After Balancing ───────────────────────────────────────────
bal_FTD, bal_CN = count_labels(balanced_meta)
print(f"[BALANCED TRAIN] total={len(balanced_meta)}  FTD={bal_FTD}, CN={bal_CN}\n")

for d in balanced_meta:
    d['label'] = label_map[d['label']]

raw_ds_train  = EEGDataset(DATA_DIR, balanced_meta)
dataset_train = BinaryEEGDataset(raw_ds_train, balanced_meta)
labels_train  = np.array([d['label'] for d in balanced_meta])

# ─── Optuna Objective ────────────────────────────────────────────
def objective(trial):
    # 1) sample hyperparameters
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    wd = trial.suggest_float('wd', 1e-6, 1e-3, log=True)


    print(f"\n--- Trial {trial.number} hyperparams --> lr={lr:.2e}, wd={wd:.2e}")

    wandb.init(
        project='eeg-optuna-FTD-CN-test-within-cross-3',
        name=f"trial{trial.number}",
        config={'lr':lr, 'wd':wd, 'pct_start':PCT_START},
        reinit=True
    )

    # train/validation split
    idx = np.arange(len(dataset_train))
    tr_idx, va_idx = train_test_split(
        idx, test_size=0.2,
        stratify=labels_train, random_state=SEED
    )

    tr_FTD = np.sum(labels_train[tr_idx] == 0)
    tr_CN = np.sum(labels_train[tr_idx] == 1)
    va_FTD = np.sum(labels_train[va_idx] == 0)
    va_CN = np.sum(labels_train[va_idx] == 1)
    print(f"[Trial {trial.number}]  "
          f"TRAIN n={len(tr_idx)} (FTD={tr_FTD}, CN={tr_CN}) | "
          f"VAL n={len(va_idx)} (FTD={va_FTD}, CN={va_CN})")

    train_loader = DataLoader(
        Subset(dataset_train, tr_idx),
        batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS
    )
    val_loader   = DataLoader(
        Subset(dataset_train, va_idx),
        batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS
    )

    # model & optimizer & scheduler & loss
    input_len = dataset_train[0][0].shape[-1]
    model = EEGformer(
        in_channels=19, input_length=input_len,
        kernel_size=10, num_filters=120,
        num_heads=3, num_blocks=1,
        num_segments=5, num_classes=2
    ).to(DEVICE)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer, max_lr=lr, epochs=MAX_EPOCHS,
        steps_per_epoch=len(train_loader), pct_start=PCT_START,
        anneal_strategy='cos', cycle_momentum=False
    )
    criterion = nn.CrossEntropyLoss()

    best_val_loss = float('inf')
    best_val_acc  = 0.0
    best_state= None
    es_count      = 0

    for epoch in range(1, MAX_EPOCHS+1):
        t0 = time.time()
        # ── train ──
        model.train()
        train_loss_sum = train_correct = train_total = 0
        for X, y in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            loss.backward()
            optimizer.step()
            scheduler.step()
            # dynamic weight_decay
            cur_lr = optimizer.param_groups[0]['lr']
            for g in optimizer.param_groups:
                g['weight_decay'] = wd * (cur_lr / lr)
            train_loss_sum += loss.item()
            preds = logits.argmax(1)
            train_correct += (preds == y).sum().item()
            train_total   += y.size(0)
        train_loss = train_loss_sum / len(train_loader)
        train_acc  = train_correct / train_total

        # ── validate ──
        model.eval()
        vloss = 0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                logits = model(X)
                vloss += criterion(logits, y).item()
                preds = logits.argmax(1)
                all_preds.append(preds.cpu().numpy())
                all_labels.append(y.cpu().numpy())
        val_loss = vloss / len(val_loader)

        preds = np.concatenate(all_preds)
        labs  = np.concatenate(all_labels)
        val_acc = (preds == labs).sum() / labs.size

        # ── metrics ──
        tn, fp, fn, tp = confusion_matrix(labs, preds, labels=[0,1]).ravel()
        spec = tn / (tn+fp) if (tn+fp)>0 else 0.0
        prec = precision_score(labs, preds, zero_division=0)
        rec  = recall_score(labs, preds, zero_division=0)
        f1   = f1_score(labs, preds, zero_division=0)
        elapsed = time.time() - t0

        print(
            f"Epoch {epoch:03d} | "
            f"train_loss={train_loss:.4f} acc={train_acc:.4f} | "
            f"val_loss={val_loss:.4f} acc={val_acc:.4f} | "
            f"prec={prec:.4f} rec={rec:.4f} spec={spec:.4f} f1={f1:.4f} | "
            f"time={elapsed:.1f}s"
        )

        wandb.log({
            'epoch':       epoch,
            'train_loss':  train_loss,
            'train_acc':   train_acc,
            'val_loss':    val_loss,
            'val_acc':     val_acc,
            'specificity': spec,
            'precision':   prec,
            'recall':      rec,
            'f1_score':    f1
        })

        # ── Early Stopping (val_loss 기준) ──
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_val_acc  = val_acc
            es_count      = 0
            best_state    = copy.deepcopy(model.state_dict())

            os.makedirs('ckpts', exist_ok=True)
            ckpt_path = f'ckpts/trial{trial.number}_optimal_FTD_CN_3.pth'
            torch.save(best_state, ckpt_path)
            trial.set_user_attr('ckpt_path', ckpt_path)
            trial.set_user_attr('best_epoch', epoch)
        else:
            es_count += 1
            if es_count >= ES_PATIENCE:
                break

    if best_state is not None:
        model.load_state_dict(best_state)

    wandb.finish()
    del model, optimizer, train_loader, val_loader, scheduler
    torch.cuda.empty_cache()

    # 다중목적 리턴
    return best_val_loss, best_val_acc

# ─── Run Optuna Study ────────────────────────────────────────────
if __name__ == "__main__":
    study = optuna.create_study(
        directions=["minimize", "maximize"],
        study_name="eeg_multiobj_FTD_CN_3"
    )
    study.optimize(objective, n_trials=10)

    # Pareto front 중 val_acc 가 가장 높은 trial 선택
    best       = max(study.best_trials, key=lambda t: t.values[1])
    bv_loss, bv_acc = best.values
    best_epoch = best.user_attrs["best_epoch"]
    ckpt_path  = best.user_attrs["ckpt_path"]

    print(f"\n=== Selected Trial #{best.number} ===")
    print(
        f" val_loss={bv_loss:.4f}, val_acc={bv_acc:.4f}, "
        f"params={best.params}, best_epoch={best_epoch}, ckpt={ckpt_path}"
    )

    # ─── Search‑model 로드 ───────────────────────────────────────
    input_len = dataset_train[0][0].shape[-1]
    search_model = EEGformer(
        in_channels=19, input_length=input_len,
        kernel_size=10, num_filters=120,
        num_heads=3, num_blocks=1,
        num_segments=5, num_classes=2
    ).to(DEVICE)

    search_model.load_state_dict(torch.load(ckpt_path, map_location=DEVICE))
    search_model.eval()
    criterion = nn.CrossEntropyLoss()

    # ─── Evaluation 함수 ─────────────────────────────────────────
    def evaluate(model, metas, criterion):
        metas_copy = copy.deepcopy(metas)
        for d in metas_copy:
            if isinstance(d["label"], str):
                d["label"] = label_map[d["label"]]

        ds = BinaryEEGDataset(EEGDataset(DATA_DIR, metas_copy), metas_copy)
        loader = DataLoader(
            ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS
        )

        model.eval()
        vloss = vcorrect = vtotal = 0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for X, y in loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                logits = model(X)
                vloss += criterion(logits, y).item()
                preds = logits.argmax(1)
                vcorrect += (preds == y).sum().item()
                vtotal += y.size(0)
                all_preds.append(preds.cpu().numpy())
                all_labels.append(y.cpu().numpy())

        preds = np.concatenate(all_preds)
        labs  = np.concatenate(all_labels)
        tn, fp, fn, tp = confusion_matrix(labs, preds, labels=[0, 1]).ravel()
        return {
            "loss":        vloss / len(loader),
            "acc":         vcorrect / vtotal,
            "sensitivity": recall_score(labs, preds, zero_division=0),
            "specificity": tn / (tn + fp) if (tn + fp) > 0 else 0.0,
            "f1":          f1_score(labs, preds, zero_division=0),
        }

    # ─── Final Evaluation on Test Sets ────────────────────────────
    print("=== Final Evaluation on Test Sets ===")
    for name, metas in [
        ("test_within", test_within_meta),
        ("test_cross",  test_cross_meta),
    ]:
        res = evaluate(search_model, metas, criterion)
        n0, n1 = count_labels(metas)
        print(f"-- {name} -- total={len(metas)}  F(TD)={n0}, C(N)={n1}")
        print(
            f" Accuracy={res['acc']:.4f} "
            f"Sensitivity={res['sensitivity']:.4f} "
            f"Specificity={res['specificity']:.4f} "
            f"F1={res['f1']:.4f}\n"
        )

[I 2025-05-08 06:54:40,632] A new study created in memory with name: eeg_multiobj_FTD_CN_3


--> Data counts before balancing:
    TRAIN         total=1831  F(TD)=729, C(N)=1102
    TEST_WITHIN   total=198  F(TD)=72, C(N)=126
    TEST_CROSS    total=554  F(TD)=247, C(N)=307

[BALANCED TRAIN] total=1458  FTD=729, CN=729


--- Trial 0 hyperparams --> lr=4.03e-05, wd=4.95e-05


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


[Trial 0]  TRAIN n=1166 (FTD=583, CN=583) | VAL n=292 (FTD=146, CN=146)
Epoch 001 | train_loss=0.7520 acc=0.4837 | val_loss=0.6931 acc=0.5000 | prec=0.5000 rec=1.0000 spec=0.0000 f1=0.6667 | time=9.1s
Epoch 002 | train_loss=0.7414 acc=0.5051 | val_loss=0.6927 acc=0.4966 | prec=0.4983 rec=0.9863 spec=0.0068 f1=0.6621 | time=10.0s
Epoch 003 | train_loss=0.7530 acc=0.5026 | val_loss=0.6925 acc=0.5000 | prec=0.5000 rec=1.0000 spec=0.0000 f1=0.6667 | time=9.9s
Epoch 004 | train_loss=0.7591 acc=0.4966 | val_loss=0.6938 acc=0.4623 | prec=0.4729 rec=0.6575 spec=0.2671 f1=0.5501 | time=10.0s
Epoch 005 | train_loss=0.7264 acc=0.5154 | val_loss=0.6928 acc=0.5000 | prec=0.5000 rec=1.0000 spec=0.0000 f1=0.6667 | time=9.9s
Epoch 006 | train_loss=0.7423 acc=0.4871 | val_loss=0.6914 acc=0.5000 | prec=0.5000 rec=1.0000 spec=0.0000 f1=0.6667 | time=9.9s
Epoch 007 | train_loss=0.7321 acc=0.5129 | val_loss=0.6908 acc=0.5034 | prec=0.5017 rec=0.9863 spec=0.0205 f1=0.6651 | time=9.8s
Epoch 008 | train_loss=

epoch,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇██
f1_score,███▇████▆██▁█▁██▁█▃██▅
precision,▇▇▇▇▇▇▇▇▇▇▇▁▇▁▇▇█▇▇▇▇█
recall,███▆████▄██▁█▁██▁█▂█▇▃
specificity,▁▁▁▃▁▁▁▁▅▁▁█▁█▁▁█▁▇▁▃▇
train_acc,▁▅▄▃▆▂▆▆█▆█▃▃█▅█▇▃▂▂▆▆
train_loss,▇▅▇█▃▆▄▃▄▃▃▃▄▃▃▁▂▃▄▂▂▂
val_acc,▅▄▅▁▅▅▅▅▅▅▅▅▄▅▅▅▅▄▄▅▆█
val_loss,▃▃▃▄▃▂▁▃▃▂▂█▂▆▂▂▅▂▄▃▂▄
epoch,22
f1_score,0.37615


...


--- Trial 9 hyperparams --> lr=1.91e-05, wd=2.22e-05


[Trial 9]  TRAIN n=1166 (FTD=583, CN=583) | VAL n=292 (FTD=146, CN=146)
Epoch 001 | train_loss=0.8204 acc=0.4940 | val_loss=0.7336 acc=0.5000 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=9.1s
Epoch 002 | train_loss=0.7933 acc=0.5000 | val_loss=0.7095 acc=0.5000 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=9.1s
Epoch 003 | train_loss=0.7913 acc=0.5086 | val_loss=0.7075 acc=0.5000 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=9.0s
Epoch 004 | train_loss=0.7708 acc=0.4940 | val_loss=0.7064 acc=0.5000 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=9.1s
Epoch 005 | train_loss=0.7645 acc=0.4966 | val_loss=0.7054 acc=0.5000 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=9.0s
Epoch 006 | train_loss=0.7589 acc=0.4889 | val_loss=0.7137 acc=0.5000 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=9.0s
Epoch 007 | train_loss=0.7705 acc=0.4691 | val_loss=0.7178 acc=0.5000 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=9.1s
Epoch 008 | train_loss=0.

epoch,▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇█████
f1_score,▁▁▁▁▁▇▇▂▄▇▄▆▆▇▆▇▇▇▇▇▇▇██████████████████
precision,▁▁▁▁▁▅▅▅▅▅▆▅▆▆▇▇▇▇▇█▇▇▇█▇▇▇█▇▇████▇▇▇▇▇▇
recall,▁▁▁▁▁▁██▁▆██▆▇▄▆█▆▅▆▆▇▅▇▇▆▆▆▆▇▆▆▇▆▆▆▆▆▆▆
specificity,███████▂▆▃▄▃▄▆▁▇▇▇▇▆▇▆▆▆▆▅▇▆▇▇▇▇▇▇▇▇▇▇▇▇
train_acc,▁▁▁▁▁▁▂▁▂▁▁▂▁▁▂▄▄▅▅▅▆▇▆▇▇▇▇▇▇▇████████▇█
train_loss,██████▇▇▇▇▇▆▆▆▅▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁
val_acc,▂▂▂▂▂▂▁▂▂▃▄▃▅▅▆▇▆▇▇▇▇█▇▇█▇███████▇██████
val_loss,█▇▇▇█▇▇▇▇▇▇▇▇▆▅▄▃▃▃▃▂▂▁▃▁▁▁▂▂▂▂▁▁▂▁▁▂▁▂▁
epoch,110
f1_score,0.77444


[I 2025-05-08 07:55:11,121] Trial 9 finished with values: [0.4187335431575775, 0.7945205479452054] and parameters: {'lr': 1.9133395390721948e-05, 'wd': 2.2206391286576762e-05}.



=== Selected Trial #7 ===
 val_loss=0.3562, val_acc=0.8596, params={'lr': 0.001135452965146807, 'wd': 7.07784025088395e-05}, best_epoch=13, ckpt=ckpts/trial7_optimal_FTD_CN_3.pth
=== Final Evaluation on Test Sets ===
-- test_within -- total=198  F(TD)=72, C(N)=126
 Accuracy=0.8434 Sensitivity=0.7619 Specificity=0.9861 F1=0.8610

-- test_cross -- total=554  F(TD)=247, C(N)=307
 Accuracy=0.6949 Sensitivity=0.5863 Specificity=0.8300 F1=0.6805



#### Train on Full Data (include validation set)
- We found out that our optimal hyperparameter is
- [lr=1.23e-03, wd=3.76e-04]
- Use this hyperparameter and include the validation set and train on entire data
- Then test the performance within and cross
- Fine tune it on our held-out validation fold

In [ ]:
# train_with_val_then_full.py
import os, json, random, time
from pathlib import Path

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, f1_score, confusion_matrix

from eeg_dataset import EEGDataset
from model_optimized_6 import EEGformer

# ─── 1) Config & reproducibility ───────────────────────────────
SEED        = 0
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

DATA_DIR    = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE  = 'labels.json'
DEVICE      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE   = 32
NUM_WORKERS  = 4
MAX_EPOCHS   = 50     # max for search
ES_PATIENCE  = 15     # early-stop patience
PCT_START    = 0.2

# hyperparameters you’ve tuned
LR           = 1.23e-3
WD           = 3.76e-4

# ─── 2) Dataset wrapper ────────────────────────────────────────
class BinaryEEGDataset(Dataset):
    def __init__(self, raw_ds, metas):
        self.raw_ds = raw_ds
        self.metas  = metas
    def __len__(self):
        return len(self.raw_ds)
    def __getitem__(self, idx):
        x, _ = self.raw_ds[idx]
        y     = self.metas[idx]['label']
        return x, y

# ─── 3) Load & prepare metadata ───────────────────────────────
with open(Path(DATA_DIR)/LABEL_FILE,'r') as f:
    all_meta = json.load(f)

train_meta       = [d for d in all_meta if d['type']=='train']
test_within_meta = [d for d in all_meta if d['type']=='test_within']
test_cross_meta  = [d for d in all_meta if d['type']=='test_cross']

# filter F vs C
class0, class1 = 'F','C'
label_map = {class0:0, class1:1}

train_meta       = [d for d in train_meta       if d['label'] in label_map]
test_within_meta = [d for d in test_within_meta if d['label'] in label_map]
test_cross_meta  = [d for d in test_cross_meta  if d['label'] in label_map]

# balance train by down-sampling
data0 = [d for d in train_meta if d['label']==class0]
data1 = [d for d in train_meta if d['label']==class1]
k     = min(len(data0), len(data1))
balanced_meta = random.sample(data0, k) + random.sample(data1, k)
random.shuffle(balanced_meta)

# convert to ints
for d in balanced_meta:       d['label'] = label_map[d['label']]
for d in test_within_meta:    d['label'] = label_map[d['label']]
for d in test_cross_meta:     d['label'] = label_map[d['label']]

print(f"\n[Balanced TRAIN] total={len(balanced_meta)} (each class={k})")

# build full balanced dataset
full_raw    = EEGDataset(DATA_DIR, balanced_meta)
full_ds     = BinaryEEGDataset(full_raw, balanced_meta)
labels_full = np.array([d['label'] for d in balanced_meta])

# ─── 4) Split train/val for epoch search ──────────────────────
idx            = np.arange(len(full_ds))
tr_idx, va_idx = train_test_split(
    idx, test_size=0.2, stratify=labels_full, random_state=SEED
)
train_loader = DataLoader(
    Subset(full_ds, tr_idx), batch_size=BATCH_SIZE,
    shuffle=True,  num_workers=NUM_WORKERS
)
val_loader   = DataLoader(
    Subset(full_ds, va_idx), batch_size=BATCH_SIZE,
    shuffle=False, num_workers=NUM_WORKERS
)

# ─── 5) Model builder & eval fn ───────────────────────────────
def build_model():
    input_len = full_ds[0][0].shape[-1]
    return EEGformer(
        in_channels=19, input_length=input_len,
        kernel_size=10, num_filters=120,
        num_heads=3, num_blocks=1,
        num_segments=5, num_classes=2
    ).to(DEVICE)

def evaluate(loader, model, criterion):
    model.eval()
    loss_sum = correct = total = 0
    with torch.no_grad():
        for X,y in loader:
            X,y    = X.to(DEVICE), y.to(DEVICE)
            logits = model(X)
            loss_sum += criterion(logits,y).item()*y.size(0)
            preds = logits.argmax(1)
            correct += (preds==y).sum().item()
            total   += y.size(0)
    return loss_sum/total, correct/total

# ─── 6) Phase 1: find best_epoch with early stopping ──────────
best_val_loss = float('inf')
best_epoch    = 0
es_count      = 0

model     = build_model()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=LR, epochs=MAX_EPOCHS,
    steps_per_epoch=len(train_loader),
    pct_start=PCT_START, anneal_strategy='cos',
    cycle_momentum=False
)
criterion = nn.CrossEntropyLoss()

print("\nSearching best epoch using validation set...")
for epoch in range(1, MAX_EPOCHS+1):
    t0 = time.time()
    # ── train ──
    model.train()
    tr_loss_sum = tr_correct = tr_total = 0
    for X,y in train_loader:
        X,y = X.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        logits = model(X)
        loss   = criterion(logits,y)
        loss.backward()
        optimizer.step()
        scheduler.step()

        tr_loss_sum  += loss.item()*y.size(0)
        preds        = logits.argmax(1)
        tr_correct  += (preds==y).sum().item()
        tr_total    += y.size(0)

    train_loss = tr_loss_sum / tr_total
    train_acc  = tr_correct / tr_total

    # ── validate ──
    val_loss, val_acc = evaluate(val_loader, model, criterion)
    elapsed = time.time() - t0

    print(
        f"Epoch {epoch:02d} | "
        f"train_loss={train_loss:.4f} | train_acc={train_acc:.4f} | "
        f"val_loss={val_loss:.4f} | val_acc={val_acc:.4f} | "
        f"time={elapsed:.1f}s"
    )

    # check for improvement
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_epoch    = epoch
        es_count      = 0
    else:
        es_count += 1
        if es_count >= ES_PATIENCE:
            print(f"\nEarly stopping at epoch {epoch} (no improvement for {ES_PATIENCE} epochs)")
            break

print(f"\n>> Best epoch by validation: {best_epoch}")

# ─── 7) Phase 2: retrain on full balanced set ─────────────────
print(f"\nRetraining from scratch on FULL set for {best_epoch} epochs...")
model2     = build_model()
optimizer2 = torch.optim.AdamW(model2.parameters(), lr=LR, weight_decay=WD)
scheduler2 = torch.optim.lr_scheduler.OneCycleLR(
    optimizer2, max_lr=LR,
    epochs=best_epoch,
    steps_per_epoch=len(DataLoader(full_ds, batch_size=BATCH_SIZE)),
    pct_start=PCT_START, anneal_strategy='cos',
    cycle_momentum=False
)
criterion2 = nn.CrossEntropyLoss()

for epoch in range(1, best_epoch+1):
    t0 = time.time()
    tr_loss_sum = tr_correct = tr_total = 0
    model2.train()
    for X,y in DataLoader(full_ds, batch_size=BATCH_SIZE,
                          shuffle=True, num_workers=NUM_WORKERS):
        X,y = X.to(DEVICE), y.to(DEVICE)
        optimizer2.zero_grad()
        logits = model2(X)
        loss   = criterion2(logits,y)
        loss.backward()
        optimizer2.step()
        scheduler2.step()

        tr_loss_sum  += loss.item()*y.size(0)
        preds        = logits.argmax(1)
        tr_correct  += (preds==y).sum().item()
        tr_total    += y.size(0)

    avg_loss = tr_loss_sum / tr_total
    acc      = tr_correct / tr_total
    print(f"Epoch {epoch:02d} | train_loss={avg_loss:.4f} acc={acc:.4f} time={time.time()-t0:.1f}s")

# ─── 8) Final Evaluation on Test Sets ─────────────────────────
def full_evaluate(metas, loader):
    model2.eval()
    loss_sum = correct = total = 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for X,y in loader:
            X,y = X.to(DEVICE), y.to(DEVICE)
            logits = model2(X)
            loss_sum += criterion2(logits,y).item()*y.size(0)
            preds    = logits.argmax(1)
            correct += (preds==y).sum().item()
            total   += y.size(0)
            all_preds.append(preds.cpu().numpy())
            all_labels.append(y.cpu().numpy())
    preds = np.concatenate(all_preds)
    labs  = np.concatenate(all_labels)
    tn,fp,fn,tp = confusion_matrix(labs,preds,labels=[0,1]).ravel()
    return {
        "loss": loss_sum/total,
        "acc": correct/total,
        "sensitivity": recall_score(labs,preds,zero_division=0),
        "specificity": tn/(tn+fp) if (tn+fp)>0 else 0.0,
        "f1": f1_score(labs,preds,zero_division=0)
    }

print("\n=== Final Evaluation on Test Sets ===")
for name, metas in [("test_within", test_within_meta), ("test_cross", test_cross_meta)]:
    loader = DataLoader(BinaryEEGDataset(EEGDataset(DATA_DIR, metas), metas),
                        batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
    res = full_evaluate(metas, loader)
    n0 = sum(1 for d in metas if d['label']==0)
    n1 = sum(1 for d in metas if d['label']==1)
    print(f"-- {name} -- total={len(metas)}  F(TD)={n0}, C(N)={n1}")
    print(f" Loss={res['loss']:.4f} Acc={res['acc']:.4f} "
          f"Sens={res['sensitivity']:.4f} Spec={res['specificity']:.4f} "
          f"F1={res['f1']:.4f}\n")



[Balanced TRAIN] total=1458 (each class=729)

Searching best epoch using validation set...
Epoch 01 | train_loss=0.7458 | train_acc=0.5026 | val_loss=0.6936 | val_acc=0.5000 | time=9.5s
Epoch 02 | train_loss=0.7211 | train_acc=0.5154 | val_loss=0.7163 | val_acc=0.5000 | time=9.2s
Epoch 03 | train_loss=0.7301 | train_acc=0.4871 | val_loss=0.6937 | val_acc=0.5000 | time=9.2s
Epoch 04 | train_loss=0.7247 | train_acc=0.4871 | val_loss=0.6888 | val_acc=0.5103 | time=9.2s
Epoch 05 | train_loss=0.6541 | train_acc=0.5935 | val_loss=0.5569 | val_acc=0.7500 | time=9.2s
Epoch 06 | train_loss=0.5958 | train_acc=0.7033 | val_loss=0.6049 | val_acc=0.6712 | time=9.1s
Epoch 07 | train_loss=0.5769 | train_acc=0.7075 | val_loss=0.5156 | val_acc=0.7671 | time=9.2s
Epoch 08 | train_loss=0.5376 | train_acc=0.7504 | val_loss=0.5084 | val_acc=0.7671 | time=9.1s
Epoch 09 | train_loss=0.4756 | train_acc=0.7762 | val_loss=0.4753 | val_acc=0.7911 | time=9.2s
Epoch 10 | train_loss=0.4194 | train_acc=0.8199 | val

#### Test performance Comparison (Without Full Train vs Full Train)

#### Without Full Train
=== Final Evaluation on Test Sets ===
- test_within -- total=198  F(TD)=72, C(N)=126
 Accuracy=0.8535 Sensitivity=0.7857 Specificity=0.9722 F1=0.8722

- test_cross -- total=554  F(TD)=247, C(N)=307
 Accuracy=0.7004 Sensitivity=0.6645 Specificity=0.7449 F1=0.7108

#### Without Full Train (Balanced Dataset)
=== Final Evaluation on Test Sets ===
- test_within -- total=198  F(TD)=72, C(N)=126
 Accuracy=0.8434 Sensitivity=0.7619 Specificity=0.9861 F1=0.8610

- test_cross -- total=554  F(TD)=247, C(N)=307
 Accuracy=0.6949 Sensitivity=0.5863 Specificity=0.8300 F1=0.6805

#### Full Train
=== Final Evaluation on Test Sets ===
- test_within -- total=198  F(TD)=72, C(N)=126
 Loss=0.3504 Acc=0.8838 Sens=0.8413 Spec=0.9583 F1=0.9021

- test_cross -- total=554  F(TD)=247, C(N)=307
 Loss=0.7021 Acc=0.6769 Sens=0.6352 Spec=0.7287 F1=0.6854

#### FTD vs CN LOSO Validation Result

In [ ]:
import os
import json
import copy
import random
import time
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import GroupKFold, GroupShuffleSplit
from sklearn.metrics import confusion_matrix

from eeg_dataset import EEGDataset
from model_optimized_6 import EEGformer

# ─── 재현성 설정 ─────────────────────────────────────────────────────
SEED = 2
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark     = False

def worker_init_fn(worker_id):
    np.random.seed(SEED + worker_id)
    random.seed(SEED + worker_id)

# ─── 설정 ─────────────────────────────────────────────────────────
DATA_DIR    = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE  = 'labels.json'
DEVICE      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE   = 32
MAX_EPOCHS   = 150   # 최대 epoch
LR           = 2.2e-05
WD           = 2.3e-06
PCT_START    = 0.2
ES_PATIENCE  = 15    # validation_loss 개선 없을 때 조기종료
VAL_SIZE     = 0.2   # train_meta 중 validation 비율

# ─── Dataset 래퍼 클래스 ──────────────────────────────────────────
class BinaryEEGDataset(Dataset):
    def __init__(self, raw_ds, meta):
        self.raw_ds = raw_ds
        self.meta   = meta
    def __len__(self):
        return len(self.raw_ds)
    def __getitem__(self, idx):
        x, _ = self.raw_ds[idx]
        y    = self.meta[idx]['label']
        return x, y

# ─── subject_id 추출 함수 ─────────────────────────────────────────
def extract_subject_id(m):
    fn = os.path.basename(m['file_name'])
    return fn.split('_')[0]

# ─── 메타데이터 로드 및 필터링 ─────────────────────────────────────
all_meta = json.load(open(os.path.join(DATA_DIR, LABEL_FILE), 'r'))
all_meta = [m for m in all_meta if m['label'] in ('F','C')]
groups   = [extract_subject_id(m) for m in all_meta]
subjects = sorted(set(groups))

# ─── LOSO 검증 셋업 ──────────────────────────────────────────────
gkf = GroupKFold(n_splits=len(subjects))
total_tp = total_tn = total_fp = total_fn = 0

# ─── Fold별 학습/평가 루프 ───────────────────────────────────────
for fold, (train_idx, test_idx) in enumerate(gkf.split(all_meta, groups=groups), 1):
    print(f"\n>>> Fold {fold}/{len(subjects)}  (leave out {subjects[fold-1]})")

    # 메타 분할 및 라벨 매핑
    train_meta = [copy.deepcopy(all_meta[i]) for i in train_idx]
    test_meta  = [copy.deepcopy(all_meta[i]) for i in test_idx]
    for m in train_meta: m['label'] = 0 if m['label']=='F' else 1
    for m in test_meta:  m['label'] = 0 if m['label']=='F' else 1

    # train_meta 안에서 validation split by subject
    groups_train = [extract_subject_id(m) for m in train_meta]
    gss = GroupShuffleSplit(n_splits=1, test_size=VAL_SIZE, random_state=SEED)
    tr_idx2, val_idx = next(gss.split(train_meta, groups=groups_train))
    val_meta   = [train_meta[i] for i in val_idx]
    train_meta = [train_meta[i] for i in tr_idx2]

    # DataLoader 준비
    raw_train = EEGDataset(DATA_DIR, train_meta)
    raw_val   = EEGDataset(DATA_DIR, val_meta)
    raw_test  = EEGDataset(DATA_DIR, test_meta)
    train_ds  = BinaryEEGDataset(raw_train, train_meta)
    val_ds    = BinaryEEGDataset(raw_val,   val_meta)
    test_ds   = BinaryEEGDataset(raw_test,  test_meta)

    train_loader = DataLoader(
        train_ds, batch_size=BATCH_SIZE, shuffle=True,
        num_workers=4, worker_init_fn=worker_init_fn
    )
    val_loader = DataLoader(
        val_ds, batch_size=BATCH_SIZE, shuffle=False,
        num_workers=4, worker_init_fn=worker_init_fn
    )
    test_loader = DataLoader(
        test_ds, batch_size=BATCH_SIZE, shuffle=False,
        num_workers=4, worker_init_fn=worker_init_fn
    )

    # 모델/옵티마이저/스케줄러/손실 함수
    input_len = train_ds[0][0].shape[-1]
    model = EEGformer(
        in_channels=19, input_length=input_len,
        kernel_size=10, num_filters=120,
        num_heads=3, num_blocks=1,
        num_segments=5, num_classes=2
    ).to(DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer, max_lr=LR, epochs=MAX_EPOCHS,
        steps_per_epoch=len(train_loader), pct_start=PCT_START,
        anneal_strategy='cos', cycle_momentum=False
    )
    criterion = nn.CrossEntropyLoss()

    # Early stopping 및 체크포인트 변수
    best_val_loss = float('inf')
    best_state    = None
    es_count      = 0

    # ─── 학습 + 검증 루프 ──────────────────────────────────────────
    for epoch in range(1, MAX_EPOCHS+1):
        t0 = time.time()
        # Train
        model.train()
        train_loss_sum = 0.0
        for X, y in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            loss.backward()
            optimizer.step()
            scheduler.step()
            # dynamic weight decay
            cur_lr = optimizer.param_groups[0]['lr']
            for g in optimizer.param_groups:
                g['weight_decay'] = WD * (cur_lr / LR)
            train_loss_sum += loss.item()
        avg_train_loss = train_loss_sum / len(train_loader)

        # Validate
        model.eval()
        val_loss_sum = val_correct = val_total = 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                logits = model(X)
                val_loss_sum += criterion(logits, y).item()
                preds = logits.argmax(1)
                val_correct += (preds == y).sum().item()
                val_total += y.size(0)
        avg_val_loss = val_loss_sum / len(val_loader)
        avg_val_acc  = val_correct / val_total if val_total>0 else 0.0
        elapsed = time.time() - t0

        print(f"  Fold {fold} Epoch {epoch}/{MAX_EPOCHS}  "
              f"Train Loss={avg_train_loss:.4f}  "
              f"Val Loss={avg_val_loss:.4f}  "
              f"Val Acc={avg_val_acc:.4f}  "
              f"Time={elapsed:.1f}s")

        # Early stopping & checkpoint on val_loss
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_state    = copy.deepcopy(model.state_dict())
            es_count      = 0
        else:
            es_count += 1
            if es_count >= ES_PATIENCE:
                print(f"  → Early stopping at epoch {epoch} (no val_loss improvement)")
                break

    # 최적의 validation 시점 모델 로드
    if best_state is not None:
        model.load_state_dict(best_state)

    # ─── 최종 테스트 & 혼동행렬 누적 ─────────────────────────────────
    all_preds, all_labels = [], []
    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            preds = model(X).argmax(1)
            all_preds.append(preds.cpu().numpy())
            all_labels.append(y.cpu().numpy())
    preds = np.concatenate(all_preds)
    labs  = np.concatenate(all_labels)
    tn, fp, fn, tp = confusion_matrix(labs, preds, labels=[0,1]).ravel()
    total_tp += tp
    total_tn += tn
    total_fp += fp
    total_fn += fn

    acc = (tp + tn) / (tp + tn + fp + fn)
    print(f"  Fold {fold} Final ACC = {acc:.4f}   (TP={tp}  TN={tn}  FP={fp}  FN={fn})")

    # 메모리 해제
    del model, optimizer, scheduler
    torch.cuda.empty_cache()

# ─── 전체 LOSO 지표 계산 ─────────────────────────────────────────
ACC  = (total_tp + total_tn) / (total_tp + total_tn + total_fp + total_fn)
SENS = total_tp / (total_tp + total_fn)
SPEC = total_tn / (total_tn + total_fp)
F1   = 2 * total_tp / (2*total_tp + total_fp + total_fn)

print("\n=== LOSO 평가 결과 ===")
print(f"Accuracy   = {ACC:.4%}")
print(f"Sensitivity= {SENS:.4%}")
print(f"Specificity= {SPEC:.4%}")
print(f"F1-score   = {F1:.4%}")

Now using CUDA device 0
Enabling CUDA with 39.14 GiB available memory

>>> Fold 1/52  (leave out sub-037)
  Fold 1 Epoch 1/150  Train Loss=0.7121  Val Loss=0.6467  Val Acc=0.6742  Time=15.0s
  Fold 1 Epoch 2/150  Train Loss=0.7085  Val Loss=0.6412  Val Acc=0.6742  Time=14.2s
  Fold 1 Epoch 3/150  Train Loss=0.7202  Val Loss=0.6421  Val Acc=0.6742  Time=14.3s
  Fold 1 Epoch 4/150  Train Loss=0.7089  Val Loss=0.6364  Val Acc=0.6742  Time=14.2s
  Fold 1 Epoch 5/150  Train Loss=0.7044  Val Loss=0.6454  Val Acc=0.6742  Time=14.2s
  Fold 1 Epoch 6/150  Train Loss=0.7103  Val Loss=0.6391  Val Acc=0.6742  Time=14.3s
  Fold 1 Epoch 7/150  Train Loss=0.7149  Val Loss=0.6419  Val Acc=0.6742  Time=14.3s
  Fold 1 Epoch 8/150  Train Loss=0.7169  Val Loss=0.6406  Val Acc=0.6742  Time=14.4s
  Fold 1 Epoch 9/150  Train Loss=0.7191  Val Loss=0.6755  Val Acc=0.6742  Time=14.5s
  Fold 1 Epoch 10/150  Train Loss=0.7088  Val Loss=0.6369  Val Acc=0.6742  Time=14.3s
  Fold 1 Epoch 11/150  Train Loss=0.7193  V